In [1]:
import os
import sys
sys.path.append("/home/jakob/Documents/PhD/Projects/ModalityRelevanceScores/code")
sys.path.append("/home/jakob/Documents/PhD/Projects/ModalityRelevanceScores/")
import numpy as np
import torch
import seaborn as sns
import pandas as pd
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline
from models.NetworkMapper import to_base_propagation, to_contribution_propagation
from datasets.SEN12MS import CLASS_NAMES, SEN12MS
from ContributionPropagation.Contribution import PerceptualScore;

/home/jakob/Documents/PhD/Projects/ModalityRelevanceScores/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Compute Statistics
def mean_var_perceptual_score(acc, n,N,K):
  mu, var = mean_var_multigeometrical(n,N,K)
  mu_perc = acc - mu / n
  var_perc = var / (n**2)
  return mu_perc, var_perc

def mean_var_multigeometrical(n,N,K):
 var = n * (K / N) * ((N - K) / N) * ((N - n) / (N - 1))
 mu = n * (K / N)

 return mu, var

In [4]:
train_cloudy = False

### Violin plots for Perceptual and RFP
plot_bias = False
setup = "All" #["Correct", "False", "All"]
seed = 42 # 43, 44:
np.random.seed(seed)
torch.random.manual_seed(seed)

root_folder = "/home/jakob/Documents/PhD/Projects/ModalityRelevanceScores/0005_SEN12MS_Train_and_Evaluate/Classification/Experiments/"

exp_name = "pretrained_original" # ResNet50_OpticalSAR_clear_seed
exp_name_cloudy = "pretrained_original" # ResNet50_OpticalSAR_cloudy_seed

# Load Scores for model trained on clear samples

if "seed" in exp_name:
    res_file_path_clear = os.path.join(root_folder, f"{exp_name}{seed}", f"results.pth")
else:
    res_file_path_clear = os.path.join(root_folder, f"{exp_name}", f"results.pth")

#perc_file_path_clear = os.path.join(root_folder, f"ResNet50_OpticalSAR_clear_seed{seed}", f"relevance_perc_score_fullFalse.pth")
#perc_clear = torch.load(perc_file_path_clear)
pred_clear = torch.load(res_file_path_clear)

if "seed" in exp_name:
    res_file_path_cloudy = os.path.join(root_folder, f"{exp_name_cloudy}{seed}", f"results.pth")
else:
    res_file_path_cloudy = os.path.join(root_folder, f"{exp_name_cloudy}", f"results.pth")
#perc_file_path_cloudy = os.path.join(root_folder, f"ResNet50_OpticalSAR_cloudy_seed{seed}", f"relevance_perc_score_fullFalse.pth")
#perc_cloudy = torch.load(perc_file_path_cloudy)
pred_cloudy = torch.load(res_file_path_cloudy)


sources = ["Bias", "SAR", "Optical"]

chosen = ["ROIs1868_summer_s2_131_p235", 
          "ROIs1970_fall_s2_141_p635.tif", 
          "ROIs1970_fall_s2_120_p292.tif",
          "ROIs1868_summer_s2_43_p264.tif"]


num_classes = pred_clear["Clear"]["Predictionss"].shape[-1]
num_samples = len(pred_cloudy["Cloudy"]["Predictionss"][0])

if train_cloudy: 
    chosen_pred = pred_cloudy
else:
    chosen_pred = pred_clear

print(f"Density over {num_samples} points!")


DATA_DIR_CLEAR = "/media/jakob/Expansion/DataSets/SEN12MS"
DATA_DIR_CLOUDY = "/media/jakob/Expansion/DataSets/SEN12MSCR"
LABEL_SPLIT_DIR = "/media/jakob/Expansion/DataSets/SEN12MS_Splits"

imgTransform = None

dataset_clear = SEN12MS(DATA_DIR_CLEAR,
                        DATA_DIR_CLOUDY, 
                        LABEL_SPLIT_DIR, 
                        img_transform=imgTransform, 
                        label_type="single_label", 
                        threshold=0.1, 
                        subset="test", 
                        use_s1=True, 
                        use_s2=True, 
                        use_rgb=True,
                        use_cloudy=False,
                        cloud_frac=0,
                        igbp_s=True)

dataset_cloudy = SEN12MS(DATA_DIR_CLEAR,
                        DATA_DIR_CLOUDY, 
                        LABEL_SPLIT_DIR, 
                        img_transform=imgTransform, 
                        label_type="single_label", 
                        threshold=0.8, 
                        subset="test", 
                        use_s1=True, 
                        use_s2=True, 
                        use_rgb=True,
                        use_cloudy=True,
                        cloud_frac=1,
                        igbp_s=True)

reps = 5

labels_str = ["Bias", "Optical", "SAR"]
for i in range(10):
    sub_ids  = [idx for idx in range(len(dataset_clear)) if dataset_clear.labels_filtered[dataset_clear.samples[idx]["id"]] == i]
    np.random.shuffle(sub_ids)
    sample_ids = sub_ids


    if len(sub_ids) == 0: continue
    fig = plt.figure(figsize=(30,5))
    gs = GridSpec(2, reps*2, figure=fig, height_ratios=[0.5,0.5], width_ratios=reps*[0.5,0.5])

    for j in range(5):

        sub_sample_id = sample_ids[j]
        

        # perc_val_1 = perc[sample_ids[j]]["orig_acc"] - perc[sample_ids[j]]["correct_mod2"] / perc[sample_ids[j]]["num_samples"]
        # perc_val_2 = perc[sample_ids[j]]["orig_acc"] - perc[sample_ids[j]]["correct_mod1"] / perc[sample_ids[j]]["num_samples"]

        data1 = dataset_clear[sub_sample_id]
        data2 = dataset_cloudy[sub_sample_id]
        sample_name = data1["id"]

        go_on = False
        if np.any(chosen_pred[i] in sample_name): go_on=True

        if not go_on: continue
        assert data1["id"] == data2["id"]

        ax11 = fig.add_subplot(gs[0:2, 2*j])
        ax13 = fig.add_subplot(gs[0, 2*j+1])        
        ax23 = fig.add_subplot(gs[1, 2*j+1])       

        img_opt_clear = data1["image"][3:0:-1].transpose([1,2,0])
        img_opt_cloudy = data2["image"][3:0:-1].transpose([1,2,0])
        img_sar = np.concatenate([data1["image"][-2:],0.5*(data1["image"][-1:]+data1["image"][-2:-1])], 0).transpose([1,2,0])
        img_sar_check = np.concatenate([data2["image"][-2:],0.5*(data2["image"][-1:]+data2["image"][-2:-1])], 0).transpose([1,2,0])

        img = np.concatenate([
            np.concatenate([img_opt_clear, np.ones([10, img_sar.shape[0],3]), img_opt_cloudy],0),
            np.ones([img_sar.shape[0]*2 + 10, 10, 3]),
            np.concatenate([np.ones_like(img_sar), np.ones([10, img_sar.shape[0],3]), img_sar],0)], 1)

        assert (img_sar == img_sar_check).all()
        # assert (img_opt_clear != img_opt_cloudy).any()
        
        # axs[i,2*j+1].axis('off')
        ax11.axis('off')
        ax11.matshow(img)
        #ax_tl.set_title(f"Label\n{data2['label']}  [{pred[:,sample_ids[i,j]].sum(0).argmax(-1).item()}]", size=12, y=1)  

        pred_pos = chosen_pred["Clear"]["SampleIDs"].index(sample_name)

        ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][0,pred_pos], label=labels_str[0], color='r')

        ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][1,pred_pos].clip(min=0), bottom=chosen_pred["Clear"]["Predictionss"][:1,pred_pos].clip(min=0).sum(0), label=labels_str[1], color='g')
        ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][1,pred_pos].clip(max=0), bottom=chosen_pred["Clear"]["Predictionss"][:1,pred_pos].clip(max=0).sum(0), color='g')

        ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][2,pred_pos].clip(min=0), bottom=chosen_pred["Clear"]["Predictionss"][:2,pred_pos].clip(min=0).sum(0), label=labels_str[2], color='b')
        ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][2,pred_pos].clip(max=0), bottom=chosen_pred["Clear"]["Predictionss"][:2,pred_pos].clip(max=0).sum(0), color='b')
        ax13.set_ylim([-20,20]) 
        ax13.set_xticks([])
    
        #ax13.set_title(f"Class {data1['label']} - Pred {pred_clear['Clear']['Predictionss'][:3,pred_pos].sum(0).argmax(-1).item()}") #  -  Perc: {perc_val_1:.2f}/{perc_val_2:.2f}")
        
        ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][0,pred_pos], label=labels_str[0], color='r')
        ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][1,pred_pos].clip(min=0), bottom=chosen_pred["Cloudy"]["Predictionss"][:1,pred_pos].clip(min=0).sum(0), label=labels_str[1], color='g')
        ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][1,pred_pos].clip(max=0), bottom=chosen_pred["Cloudy"]["Predictionss"][:1,pred_pos].clip(max=0).sum(0), color='g')

        ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][2,pred_pos].clip(min=0), bottom=chosen_pred["Cloudy"]["Predictionss"][:2,pred_pos].clip(min=0).sum(0), label=labels_str[2], color='b')
        ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][2,pred_pos].clip(max=0), bottom=chosen_pred["Cloudy"]["Predictionss"][:2,pred_pos].clip(max=0).sum(0), color='b')
        ax23.set_ylim([-20,20]) 
        ax23.set_xticks(range(10))
        
        # ax23.set_title(f"Class {data1['label']} - Pred {pred_clear['Clear']['Predictionss'][:3,pred_pos].sum(0).argmax(-1).item()}") #  -  Perc: {perc_val_1:.2f}/{perc_val_2:.2f}")
        

        #ax_b.legend(fontsize=10, title_fontsize=15)
        #ax23.subplots_adjust(hspace=0.0)  # Adjust the vertical spacing between subplots
        ax11.axis('off')
        ax11.legend(*ax13.get_legend_handles_labels(), fontsize=10, loc="upper right")
        ax11.set_title(f"Trained Cloudy: {train_cloudy}" +\
                       f"\nClass {CLASS_NAMES[i]}" + \
                    f"\nClear: {CLASS_NAMES[chosen_pred['Clear']['Predictionss'].sum(0)[pred_pos].argmax(-1).item()]}" + \
                    f"\n Cloudy:  {CLASS_NAMES[chosen_pred['Cloudy']['Predictionss'].sum(0)[pred_pos].argmax(-1).item()]}")
        #plt.tight_layout()
        #plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=1, wspace=None, hspace=None)

    fig.tight_layout()
    plt.show()


Density over 18106 points!
path clear:  /media/jakob/Expansion/DataSets/SEN12MS
path cloudy:  /media/jakob/Expansion/DataSets/SEN12MSCR


[Load]: 100%|██████████| 18106/18106 [00:00<00:00, 140557.45it/s]

loaded 18106 samples from the sen12ms subset test


path clear:  /media/jakob/Expansion/DataSets/SEN12MS
path cloudy:  /media/jakob/Expansion/DataSets/SEN12MSCR


[Load]: 100%|██████████| 18106/18106 [00:00<00:00, 96698.74it/s]

Loaded 18106 files with 12666 Cloudy versions!
loaded 18106 samples from the sen12ms subset test


KeyError: 0

<Figure size 3000x500 with 0 Axes>

In [31]:
train_cloudy = False
data_avail = False
### Violin plots for Perceptual and RFP
plot_bias = False
setup = "All" #["Correct", "False", "All"]
seed = 42 # 43, 44:
np.random.seed(seed)
torch.random.manual_seed(seed)

root_folder = "/home/jakob/Documents/PhD/Projects/ModalityRelevanceScores/0005_SEN12MS_Train_and_Evaluate/Classification/Experiments/"

exp_name = "pretrained_original" # ResNet50_OpticalSAR_clear_seed
exp_name_cloudy = "pretrained_original" # ResNet50_OpticalSAR_cloudy_seed

# Load Scores for model trained on clear samples

if "seed" in exp_name:
    res_file_path_clear = os.path.join(root_folder, f"{exp_name}{seed}", f"results.pth")
else:
    res_file_path_clear = os.path.join(root_folder, f"{exp_name}", f"results.pth")

perc_file_path_clear = os.path.join(root_folder, f"pretrained_original", f"relevance_perc_score_fullFalse.pth")
perc_clear = torch.load(perc_file_path_clear)
pred_clear = torch.load(res_file_path_clear)

if "seed" in exp_name:
    res_file_path_cloudy = os.path.join(root_folder, f"{exp_name_cloudy}{seed}", f"results.pth")
else:
    res_file_path_cloudy = os.path.join(root_folder, f"{exp_name_cloudy}", f"results.pth")
perc_file_path_cloudy = os.path.join(root_folder, f"pretrained_original", f"relevance_perc_score_fullFalse.pth")
perc_cloudy = torch.load(perc_file_path_cloudy)
pred_cloudy = torch.load(res_file_path_cloudy)


sources = ["Bias", "SAR", "Optical"]

num_classes = pred_clear["Clear"]["Predictionss"].shape[-1]
num_samples = len(pred_cloudy["Cloudy"]["Predictionss"][0])

if train_cloudy: 
    chosen_pred = pred_cloudy
else:
    chosen_pred = pred_clear

print(f"Density over {num_samples} points!")


DATA_DIR_CLEAR = "/media/jakob/Expansion/DataSets/SEN12MS"
DATA_DIR_CLOUDY = "/media/jakob/Expansion/DataSets/SEN12MSCR"
LABEL_SPLIT_DIR = "/media/jakob/Expansion/DataSets/SEN12MS_Splits"

imgTransform = None



dataset_clear = SEN12MS(DATA_DIR_CLEAR,
                        DATA_DIR_CLOUDY, 
                        LABEL_SPLIT_DIR, 
                        img_transform=imgTransform, 
                        label_type="single_label", 
                        threshold=0.1, 
                        subset="test", 
                        use_s1=True, 
                        use_s2=True, 
                        use_rgb=True,
                        use_cloudy=False,
                        cloud_frac=0,
                        igbp_s=True)

dataset_cloudy = SEN12MS(DATA_DIR_CLEAR,
                        DATA_DIR_CLOUDY, 
                        LABEL_SPLIT_DIR, 
                        img_transform=imgTransform, 
                        label_type="single_label", 
                        threshold=0.8, 
                        subset="test", 
                        use_s1=True, 
                        use_s2=True, 
                        use_rgb=True,
                        use_cloudy=True,
                        cloud_frac=1,
                        igbp_s=True)


base_folder = "Example_Images_SEN12MS"
os.makedirs(base_folder, exist_ok=True)


max_val = 2000

target_folder = os.path.join("Example_Images_SEN12MS", f"Uploads")
os.makedirs(target_folder, exist_ok=True)    
labels_str = ["Bias", "SAR", "Optical"]
for sub_sample_id in range(len(dataset_cloudy)):

    s = dataset_clear.samples[sub_sample_id]["id"]
    if s not in ["ROIs1868_summer_s2_131_p235.tif",
                    "ROIs1868_summer_s2_131_p235.tif",
                    "ROIs1970_fall_s2_141_p635.tif",
                    "ROIs1970_fall_s2_120_p292.tif", 
                    "ROIs1868_summer_s2_43_p264.tif"]: continue

    perc_val_1_clear = perc_clear["Clear"][s]["Accuracy"] - perc_clear["Clear"][s]["Correct SAR"] / perc_clear["Clear"][s]["Total Count"]
    perc_val_2_clear = perc_clear["Clear"][s]["Accuracy"] - perc_clear["Clear"][s]["Correct Opt"] / perc_clear["Clear"][s]["Total Count"]
    perc_val_1_cloudy = perc_val_1_clear
    perc_val_2_cloudy = perc_val_2_clear
    

    data1 = dataset_clear[sub_sample_id]
    data2 = dataset_cloudy[sub_sample_id]
    sample_name = data1["id"]

    assert data1["id"] == data2["id"]

    fig = plt.figure(figsize=(10,8))
    gs = GridSpec(2, 2, figure=fig, height_ratios=[0.5,0.5], width_ratios=[0.5,0.5])
    sub_sample_id = sample_ids[j]

    ax11 = fig.add_subplot(gs[0:2, 0])
    ax13 = fig.add_subplot(gs[0, 1])        
    ax23 = fig.add_subplot(gs[1, 1])       

    img_opt_clear = np.clip(data1["image"][3:0:-1].transpose([1,2,0]), 0, max_val) / max_val
    img_opt_cloudy = np.clip(data2["image"][3:0:-1].transpose([1,2,0]), 0, max_val) / max_val
    img_sar = np.concatenate([data1["image"][-2:],0.5*(data1["image"][-1:]+data1["image"][-2:-1])], 0).transpose([1,2,0])
    img_sar_check = np.concatenate([data2["image"][-2:],0.5*(data2["image"][-1:]+data2["image"][-2:-1])], 0).transpose([1,2,0])

    img = np.concatenate([
        np.concatenate([img_opt_clear, np.ones([10, img_sar.shape[0],3]), img_opt_cloudy],0),
        np.ones([img_sar.shape[0]*2 + 10, 10, 3]),
        np.concatenate([np.ones_like(img_sar), np.ones([10, img_sar.shape[0],3]), img_sar],0)], 1)

    assert (img_sar == img_sar_check).all()
    # assert (img_opt_clear != img_opt_cloudy).any()
    
    # axs[i,2*j+1].axis('off')
    ax11.axis('off')
    ax11.matshow(img)
    #ax_tl.set_title(f"Label\n{data2['label']}  [{pred[:,sample_ids[i,j]].sum(0).argmax(-1).item()}]", size=12, y=1)  

    pred_pos = chosen_pred["Clear"]["SampleIDs"].index(sample_name)

    text_size = 18
    ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][0,pred_pos], label=labels_str[0], color='r')
    ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][1,pred_pos].clip(min=0), width=0.8, bottom=chosen_pred["Clear"]["Predictionss"][:1,pred_pos].clip(min=0).sum(0), label=labels_str[1], color='g')
    ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][1,pred_pos].clip(max=0), width=0.8, bottom=chosen_pred["Clear"]["Predictionss"][:1,pred_pos].clip(max=0).sum(0), color='g')
    ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][2,pred_pos].clip(min=0), width=0.8, bottom=chosen_pred["Clear"]["Predictionss"][:2,pred_pos].clip(min=0).sum(0), label=labels_str[2], color='b')
    ax13.bar(range(10), chosen_pred["Clear"]["Predictionss"][2,pred_pos].clip(max=0), width=0.8, bottom=chosen_pred["Clear"]["Predictionss"][:2,pred_pos].clip(max=0).sum(0), color='b')
    ax13.bar(np.arange(10)+0.2, chosen_pred["Clear"]["Predictionss"][:3,pred_pos].sum(0), label="Output", width=0.4, color='gray', alpha=0.85, linestyle="--", edgecolor="black")
    ax13.hlines(y=0,xmin=-0.5, xmax=9.5, color="black")

    ax13.set_ylim([-40,40]) 
    ax13.tick_params(axis='x', labelsize=text_size)
    ax13.tick_params(axis='y', labelsize=text_size)
    ax13.set_xticks(range(10), CLASS_NAMES, rotation=45, ha="right", fontsize=text_size)

    # ax13.title(f"Class {CLASS_NAMES[label]}  -  Prediction {pred[:3,sub_sample_id].sum(0).argmax(-1).item()}  -  Perc. Scores: {perc_val_1:.2f} / {perc_val_2:.2f}", fontsize=text_size)
    #ax13.legend(fontsize=text_size, title_fontsize=text_size)



    text_size = 18
    ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][0,pred_pos], label=labels_str[0], color='r')
    ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][1,pred_pos].clip(min=0), width=0.8, bottom=chosen_pred["Cloudy"]["Predictionss"][:1,pred_pos].clip(min=0).sum(0), label=labels_str[1], color='g')
    ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][1,pred_pos].clip(max=0), width=0.8, bottom=chosen_pred["Cloudy"]["Predictionss"][:1,pred_pos].clip(max=0).sum(0), color='g')
    ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][2,pred_pos].clip(min=0), width=0.8, bottom=chosen_pred["Cloudy"]["Predictionss"][:2,pred_pos].clip(min=0).sum(0), label=labels_str[2], color='b')
    ax23.bar(range(10), chosen_pred["Cloudy"]["Predictionss"][2,pred_pos].clip(max=0), width=0.8, bottom=chosen_pred["Cloudy"]["Predictionss"][:2,pred_pos].clip(max=0).sum(0), color='b')
    ax23.bar(np.arange(10)+0.2, chosen_pred["Cloudy"]["Predictionss"][:3,pred_pos].sum(0), label="Output", width=0.4, color='gray', alpha=0.85, linestyle="--", edgecolor="black")
    ax23.hlines(y=0,xmin=-0.5, xmax=9.5, color="black")
    ax23.tick_params(axis='x', labelsize=text_size)
    ax23.tick_params(axis='y', labelsize=text_size)
    ax23.set_ylim([-40,40]) 
    ax23.set_xticks(range(10), CLASS_NAMES, rotation=45, ha="right", fontsize=text_size)

    # ax23.title(f"Class {CLASS_NAMES[label]}  -  Prediction {pred[:3,sub_sample_id].sum(0).argmax(-1).item()}  -  Perc. Scores: {perc_val_1:.2f} / {perc_val_2:.2f}", fontsize=text_size)
    #ax23.legend(fontsize=text_size, title_fontsize=text_size)
    


    ax11.axis('off')
    ax11.legend(*ax13.get_legend_handles_labels(), fontsize=text_size, loc="upper right")
    ax11.set_title(f"\nTrue Class: {CLASS_NAMES[i]}" + \
                f"\nClear Prediction: {CLASS_NAMES[chosen_pred['Clear']['Predictionss'].sum(0)[pred_pos].argmax(-1).item()]}" + \
                f"\nCloudy Prediction:  {CLASS_NAMES[chosen_pred['Cloudy']['Predictionss'].sum(0)[pred_pos].argmax(-1).item()]}",
                fontsize=text_size)
    

    #ax11.set_title(f"Perc. Scores: {perc_val_1_clear} / {perc_val_2_clear}\n" +\
    #                f"Perc. Scores: {perc_val_1_cloudy} / {perc_val_1_cloudy}",
    #            fontsize=text_size)
    
    
    #plt.tight_layout()
    #plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=1, wspace=None, hspace=None)

    fig.tight_layout()
    plt.savefig(os.path.join(target_folder, f"{data1['id']}.png"), dpi=180)
    plt.clf()
    plt.close()


Density over 18106 points!
path clear:  /media/jakob/Expansion/DataSets/SEN12MS
path cloudy:  /media/jakob/Expansion/DataSets/SEN12MSCR


[Load]: 100%|██████████| 18106/18106 [00:00<00:00, 106750.17it/s]

loaded 18106 samples from the sen12ms subset test


path clear:  /media/jakob/Expansion/DataSets/SEN12MS
path cloudy:  /media/jakob/Expansion/DataSets/SEN12MSCR


[Load]: 100%|██████████| 18106/18106 [00:00<00:00, 54054.22it/s]


Loaded 18106 files with 12666 Cloudy versions!
loaded 18106 samples from the sen12ms subset test
